# Bucketing & Partitioning

* see the advantages of these techniques
* join users with questions
 * take questions only for the year 2018
 * see the query plan

In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc

import os

In [2]:
spark = (
    SparkSession
    .builder
    .appName('Bucketing II')
    .enableHiveSupport()
    .getOrCreate()
)

In [6]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-2]) 

questions_input_path = os.path.join(project_path, 'output/1/questions-partitioned')

users_input_path = os.path.join(project_path, 'output/users-bucketed')

In [4]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|    users|      false|
+--------+---------+-----------+



In [7]:
usersDF = spark.table('users')

questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
)

In [21]:
usersDF.orderBy(desc('user_id')).show(1000)

+-------+--------------------+--------------------+--------------------+---------+-------+----------+-----+
|user_id|        display_name|               about|            location|downvotes|upvotes|reputation|views|
+-------+--------------------+--------------------+--------------------+---------+-------+----------+-----+
| 214760|                 K m|                null|                null|        0|      0|         1|    0|
| 214759|              Mithra|                null|                null|        0|      0|         1|    0|
| 214758|               Jack |                null|                null|        0|      0|         1|    0|
| 214757|      Sabyasachi Dey|                null|                null|        0|      0|         1|    0|
| 214756|       Pearson Frank|                null|                null|        0|      0|         1|    0|
| 214755|              John B|                null|                null|        0|      0|         1|    0|
| 214754|          675453436

In [22]:
(
    questionsDF
    #.filter(col('year').isin([2018, 2017, 2016]))
    .repartition(10, 'user_id')
    .join(usersDF, 'user_id')
    .filter(col('user_id').isin([214760, 214757, 214741, 214748]))
).show()

+-------+-----------+----+-------------+-----+----+------------------+-------+--------+-----+----+------------+-----+--------+---------+-------+----------+-----+
|user_id|question_id|tags|creation_date|title|body|accepted_answer_id|answers|comments|views|year|display_name|about|location|downvotes|upvotes|reputation|views|
+-------+-----------+----+-------------+-----+----+------------------+-------+--------+-----+----+------------+-----+--------+---------+-------+----------+-----+
+-------+-----------+----+-------------+-----+----+------------------+-------+--------+-----+----+------------+-----+--------+---------+-------+----------+-----+

